# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails

from train import clean_data
import os
import pandas as pd
from train import clean_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
import joblib

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#ws = Workspace.from_config()
ws = Workspace.get(name="quick-starts-ws-132278")

experiment_name = 'hyperdrive'

experiment=Experiment(ws, experiment_name)

In [3]:
#  Create compute cluster

cluster_name = "cpu-cluster-1"
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-12-29T08:30:38.244000+00:00', 'errors': None, 'creationTime': '2020-12-29T07:59:42.632975+00:00', 'modifiedTime': '2020-12-29T07:59:58.474047+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
%%writefile conda_dependencies_1.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Overwriting conda_dependencies_1.yml


In [5]:
# Env needed on ACI
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies_1.yml')

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
                    {
                    '--C': choice(0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 10, 100),
                    '--max_iter': choice(100, 200, 300)
                    }
                )

#TODO: Create your estimator and hyperdrive config
#if "training" not in os.listdir():
#    os.mkdir("./training")
#import shutil
#project_folder = "./training"
#shutil.copy('train.py', project_folder)
project_folder = "./"
estimator = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      #arguments=['--C', '1.0', '--max_iter', 100],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=15,
                                     max_concurrent_runs=4)

In [7]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3662c5eb-a589-4899-9734-475bb5c399c7
Web View: https://ml.azure.com/experiments/hyperdrive/runs/HD_3662c5eb-a589-4899-9734-475bb5c399c7?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-132278/workspaces/quick-starts-ws-132278

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-29T08:49:05.581214][API][INFO]Experiment created<END>\n""<START>[2020-12-29T08:49:06.730446][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-29T08:49:07.047317][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-29T08:49:06.8903204Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print("Best run details:", best_run.get_details()['runDefinition']['arguments'])
print("Best run id:", best_run.id)
print("Best run accuracy:", best_run_metrics['Accuracy'])
print("Best run files:", best_run.get_file_names())


In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='hyperdrive', model_path='outputs/model-hyperdrive.joblib')
model.download(target_dir="outputs", exist_ok=True)

In [ ]:
# Evaluate on val set
data_file = "https://raw.githubusercontent.com/shbv/azure_ml/main/capstone/titanic_data.csv"
ds = Dataset.Tabular.from_delimited_files(data_file)

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
model = joblib.load('outputs/model-hyperdrive.joblib')
print("Accuracy:", accuracy_score(y_test, model.predict(x_test)))
print("Classification report:")
print(classification_report(y_test, model.predict(x_test)))
print("Confusion matrix:")
print(confusion_matrix(y_test, model.predict(x_test)))

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service